# LangChain Basics

## 1. Setup

In [1]:
# Import required modules
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Load environment variables
load_dotenv()

# Model name
MODEL_NAME = "gpt-5-mini"

## 2. Calling a Language Model

In [2]:
# Create the model
model = ChatOpenAI(model=MODEL_NAME)

# Set the question (prompt)
user_prompt = "What are the key points to keep in mind when using a language model?"
messages = [HumanMessage(content=user_prompt)]

# Call the language model
response = model.invoke(messages)

# Display the result
print(response.content)

Briefly — treat a language model as a powerful, flexible assistant, not an oracle. Key points to keep in mind:

1. Purpose & expectations
   - Models are excellent at generating text, summarizing, rewriting, brainstorming, and following instructions, but they do not "know" in a human sense and can be confidently wrong.

2. Hallucinations and factual errors
   - Models can invent facts, references, or quotes. Always verify critical information against trusted sources.

3. Limitations of knowledge and currency
   - A model’s training cut‑off or update schedule means it may lack recent facts or changing context. Check dates and version info.

4. Bias and fairness
   - Outputs can reflect training data biases. Anticipate and check for biased, offensive, or discriminatory content, especially in sensitive domains.

5. Prompt design matters
   - Clear, specific prompts with required format, constraints, and examples (few‑shot) produce better results. Use system/role framing when available.

6

## 3. Setting System Prompts and Parameters

In [3]:
# Create the model
model = ChatOpenAI(
    reasoning_effort="low",
    model=MODEL_NAME
)

# Set the prompts
system_prompt = "You are an elementary school teacher. You answer clearly and concisely, using examples."
user_prompt = "What are the key points to keep in mind when using a language model?"
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

# Call the language model and display the result (streaming)
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)


Here are the key points to remember when using a language model, written simply with short examples:

1. Models predict text, not facts
   - They guess the next words based on patterns in their training data. Their answers can be wrong or made up.
   - Example: Ask “When did X happen?” and always double-check the date with a trusted source.

2. Be clear and specific in your prompt
   - Better input gives better output. Say what you want, the format, and any constraints.
   - Example: Instead of “Write a story,” say “Write a 150‑word story for 8‑year‑olds about a brave kitten.”

3. Verify important information
   - Check facts, numbers, medical or legal advice with reliable sources or experts.
   - Example: Use the model for a draft of health advice, then confirm details with a doctor or official guidance.

4. Watch for bias and unfairness
   - Models reflect biases in their training data. They can produce stereotyped or unfair content.
   - Example: If a model gives a biased descriptio

## 4. Prompt Templates

In [4]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are an excellent translator who translates from {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template),
])

# Example: English -> Spanish
messages = chat_prompt.format_messages(
    input_language="English",
    output_language="Spanish",
    text="I love programming."
)

# The generated prompt messages
messages


[SystemMessage(content='You are an excellent translator who translates from English to Spanish.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]

In [5]:
# Use the prompt we created
# Create the model
model = ChatOpenAI(model=MODEL_NAME)

# Call the language model
response = model.invoke(messages)

# Display the result
print(response.content)

Me encanta programar.


## 5. Output Parser

In [6]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# Create the model
model = ChatOpenAI(model=MODEL_NAME)

# Set the question
user_prompt = "Please output 10 English words that start with 'a', separated by commas."
messages = [HumanMessage(content=user_prompt)]

# Call the language model
response = model.invoke(messages)

# Create an Output Parser
output_parser = CommaSeparatedListOutputParser()

# Parse with the Output Parser
word_list = output_parser.parse(response.content)
print(type(word_list))
print(word_list)


<class 'list'>
['apple', 'art', 'animal', 'answer', 'ability', 'amazing', 'accept', 'active', 'achieve', 'adapt']


## 6. Structured Output from Language Models

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

# 1. Define the desired output data structure (schema) with Pydantic
class WordList(BaseModel):
    words: List[str] = Field(description="a list of words starting with 'a'")

# 2. Create the model
model = ChatOpenAI(model=MODEL_NAME)  # A Structured Output–capable model is recommended

# 3. Force structured output from the model (OpenAI parameters are set here)
structured_llm = model.with_structured_output(WordList)

# 4. Run
user_prompt = "Please output 10 English words that start with 'a'."
response = structured_llm.invoke(user_prompt)

# Check the result
print(f"Type: {type(response.words)}")
print(f"List: {response.words}")


Type: <class 'list'>
List: ['apple', 'angle', 'answer', 'animal', 'arrival', 'abstract', 'abundant', 'attempt', 'antelope', 'allocate']


In [8]:
from langchain_openai import ChatOpenAI

# 1. Define the desired output structure as a JSON Schema (dictionary)
json_schema = {
    "title": "WordList",
    "description": "a list of words starting with 'a'",
    "type": "object",
    "properties": {
        "words": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of words"
        }
    },
    "required": ["words"]
}

# 2. Create the model
model = ChatOpenAI(model=MODEL_NAME)

# 3. Create a structured-output model with method="json_mode"
# This makes the return value a plain dict instead of a Pydantic object
structured_llm = model.with_structured_output(json_schema, method="json_mode")

# 4. Run
# IMPORTANT: When using json_mode, clearly instruct the model to return JSON in the prompt
user_prompt = "Please output 5 English words that start with 'a' in JSON format."
response = structured_llm.invoke(user_prompt)

# Check the result
print(f"Type: {type(response)}")
print(f"Dict: {response}")


Type: <class 'dict'>
Dict: {'words': ['apple', 'ant', 'anchor', 'artist', 'answer']}
